In [74]:
import requests, sys
import pandas as pd
import numpy as np
import ast
import csv

with open('ensemble_metazoa.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))

FileNotFoundError: [Errno 2] No such file or directory: 'ensemble_metazoa.csv'